In [36]:
'''
ideas to improve:
-train more epochs
-more training data
-different architecture
-different input size
-different batch size



'''

'\nideas to improve:\n-train more epochs\n-more training data\n-different architecture\n-different input size\n-different batch size\n\n\n\n'

In [37]:
import string
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout, Activation, Bidirectional
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
import numpy as np
from keras.models import load_model


# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# load document
in_filename = 'cleanedespn.txt'
doc = load_doc(in_filename)
print(doc[:200])

#grab subset
#print(len(doc))
#doc = doc[:700000]

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'espnword.txt'
save_doc(sequences, out_filename)

If the rumors are true and Doc Rivers gets a contract extension soon, I just hope the Celtics tell the truth in the ensuing news conference. Don't feed us lines like "Thanks to Doc, our young players 
['if', 'the', 'rumors', 'are', 'true', 'and', 'doc', 'rivers', 'gets', 'a', 'contract', 'extension', 'soon', 'i', 'just', 'hope', 'the', 'celtics', 'tell', 'the', 'truth', 'in', 'the', 'ensuing', 'news', 'conference', 'dont', 'feed', 'us', 'lines', 'like', 'thanks', 'to', 'doc', 'our', 'young', 'players', 'made', 'major', 'strides', 'this', 'season', 'or', 'we', 'would', 'have', 'made', 'the', 'playoffs', 'if', 'we', 'didnt', 'have', 'so', 'many', 'injuries', 'thats', 'a', 'load', 'of', 'crap', 'maybe', 'those', 'arent', 'lies', 'but', 'they', 'seem', 'like', 'fibs', 'along', 'the', 'lines', 'of', 'i', 'dont', 'think', 'those', 'jeans', 'make', 'you', 'look', 'fat', 'at', 'all', 'theyre', 'just', 'the', 'wrong', 'size', 'and', 'i', 'wasnt', 'checking', 'her', 'out', 'i', 'thought', 'i', '

In [2]:
MIN_WORD_FREQUENCY = 5

word_freq = {}
for word in tokens:
    word_freq[word] = word_freq.get(word, 0) + 1

ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

In [3]:
words = set(tokens)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

Unique words before ignoring: 39395
Ignoring words with frequency < 5
Unique words after ignoring: 10798


In [4]:
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

In [5]:
SEQUENCE_LEN = 25
STEP = 1

sentences = []
next_words = []
ignored = 0
for i in range(0, len(tokens) - SEQUENCE_LEN, STEP):
    # Only add the sequences where no word is in ignored_words
    if len(set(tokens[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(tokens[i: i + SEQUENCE_LEN])
        next_words.append(tokens[i + SEQUENCE_LEN])
    else:
        ignored = ignored + 1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Ignored sequences: 644670
Remaining sequences: 371796


In [6]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
        y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t, word_indices[w]] = 1
            y[i, word_indices[next_word_list[index % len(sentence_list)]]] = 1
            index = index + 1
        yield x, y

In [7]:
def get_model(dropout=0.2):
    print('Build model...')
    model = Sequential()
    model.add(Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, len(words))))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    return model

In [8]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)

In [9]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)

    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()

In [10]:
(sentences, next_words), (sentences_test, next_words_test) = shuffle_and_split_training_set(sentences, next_words)

Shuffling sentences
Size of training set = 364360
Size of test set = 7436


In [35]:
model = get_model()
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

Build model...


In [38]:
file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % (
        len(words),
        SEQUENCE_LEN,
        MIN_WORD_FREQUENCY
    )
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]

In [ ]:
BATCH_SIZE = 32

model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                    epochs=50,
                    validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                    validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

Epoch 1/50
11387/11387 [==============================] - 1220s 107ms/step - loss: 5.1500 - acc: 0.1675 - val_loss: 5.5173 - val_acc: 0.1540
Epoch 2/50
11387/11387 [==============================] - 1215s 107ms/step - loss: 4.7744 - acc: 0.1948 - val_loss: 5.6011 - val_acc: 0.1553
Epoch 3/50
11387/11387 [==============================] - 1216s 107ms/step - loss: 4.4011 - acc: 0.2267 - val_loss: 5.7733 - val_acc: 0.1448
Epoch 4/50
11387/11387 [==============================] - 1216s 107ms/step - loss: 4.0363 - acc: 0.2633 - val_loss: 5.9736 - val_acc: 0.1377
Epoch 5/50
11387/11387 [==============================] - 1215s 107ms/step - loss: 3.6980 - acc: 0.3036 - val_loss: 6.1341 - val_acc: 0.1293
Epoch 6/50
11387/11387 [==============================] - 1215s 107ms/step - loss: 3.3779 - acc: 0.3467 - val_loss: 6.3519 - val_acc: 0.1239
Epoch 7/50
 5042/11387 [============>.................] - ETA: 11:07 - loss: 3.1575 - acc: 0.3768

In [ ]:
model.save('modelv2.h5')

In [16]:
# load
in_filename = 'espnword.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 50)            629350    
_________________________________________________________________
lstm_5 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 12587)             1271287   
Total params: 2,051,537
Trainable params: 2,051,537
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
117185/117185 [==============================] - 147s 1ms/step - loss: 7.2820 - acc: 0.0551
Epoch 2/100
117185/1171

In [13]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'espnword.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('modelv2.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))


# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

work or maybe we can package some of these guys for a superstar so theres a little bit of hope there even if its misguided ridiculous and inane when i was there no hope whatsoever and that was my biggest mistakesimmons so you like what isiah has donelayden hell yeah take



ValueError: Error when checking input: expected bidirectional_1_input to have 3 dimensions, but got array with shape (1, 50)

In [12]:
seed_text = "tom"
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

humping crisp defeats bomb and phillips jim importantly bout the best sports movies of alltime the biggest stretch in the draft that morrisons suit when flair flushed us he was involved to be the first of the way i understand what you can see the word upside in the first


In [16]:
model = load_model('modelv2.h5')
model.weights

[<tf.Variable 'bidirectional_1_3/forward_lstm_1/kernel:0' shape=(10798, 512) dtype=float32_ref>,
 <tf.Variable 'bidirectional_1_3/forward_lstm_1/recurrent_kernel:0' shape=(128, 512) dtype=float32_ref>,
 <tf.Variable 'bidirectional_1_3/forward_lstm_1/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'bidirectional_1_3/backward_lstm_1/kernel:0' shape=(10798, 512) dtype=float32_ref>,
 <tf.Variable 'bidirectional_1_3/backward_lstm_1/recurrent_kernel:0' shape=(128, 512) dtype=float32_ref>,
 <tf.Variable 'bidirectional_1_3/backward_lstm_1/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'dense_1_3/kernel:0' shape=(256, 10798) dtype=float32_ref>,
 <tf.Variable 'dense_1_3/bias:0' shape=(10798,) dtype=float32_ref>]

In [18]:
word_indices

{'a': 0,
 'aampm': 1,
 'aaron': 2,
 'aba': 3,
 'abandoned': 4,
 'abbie': 5,
 'abc': 6,
 'abcs': 7,
 'abdurrahim': 8,
 'aberration': 9,
 'abilities': 10,
 'ability': 11,
 'able': 12,
 'aboard': 13,
 'about': 14,
 'above': 15,
 'aboveaverage': 16,
 'abreu': 17,
 'abruptly': 18,
 'absence': 19,
 'absolute': 20,
 'absolutely': 21,
 'absorb': 22,
 'absorbed': 23,
 'absorbing': 24,
 'absurd': 25,
 'abuse': 26,
 'abysmal': 27,
 'ac': 28,
 'academy': 29,
 'accent': 30,
 'accents': 31,
 'accept': 32,
 'acceptable': 33,
 'acceptance': 34,
 'accepted': 35,
 'accepting': 36,
 'accepts': 37,
 'access': 38,
 'accident': 39,
 'accidentally': 40,
 'accidents': 41,
 'accompanied': 42,
 'accompanying': 43,
 'accomplish': 44,
 'accomplished': 45,
 'accomplishment': 46,
 'according': 47,
 'accordingly': 48,
 'account': 49,
 'accountable': 50,
 'accounts': 51,
 'accuracy': 52,
 'accurate': 53,
 'accused': 54,
 'ace': 55,
 'aces': 56,
 'achievement': 57,
 'acid': 58,
 'acie': 59,
 'acknowledge': 60,
 'acl':

In [20]:
model.predict(2)

AttributeError: 'int' object has no attribute 'ndim'

In [35]:
SEQUENCE_LEN = 25

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

model = load_model('modelv2.h5')

seed_index = np.random.randint(len(sentences+sentences_test))
seed = (sentences+sentences_test)[seed_index]
#seed = ['the', 'best', 'player', 'in', 'the', 'world', 'is']


'''
for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed

        
        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)
        
        print(seed)    
        print(sentence)
'''

diversity = 1
sentence = seed

print(seed)  
for i in range(50):
    x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
    for t, word in enumerate(sentence):
        x_pred[0, t, word_indices[word]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_word = indices_word[next_index]

    sentence = sentence[1:]
    print(sentence)
    sentence.append(next_word)
    
print(sentence)
 
for i in range(50):
    x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
    for t, word in enumerate(sentence):
        x_pred[0, t, word_indices[word]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_word = indices_word[next_index]

    sentence = sentence[1:]
    sentence.append(next_word)
    

  
print(sentence)

['pr', 'people', 'running', 'the', 'events', 'hope', 'i', 'might', 'write', 'about', 'them', 'not', 'because', 'im', 'actually', 'a', 'celebrity', 'i', 'left', 'that', 'morning', 'dressed', 'in', 'casual', 'clothes']
['people', 'running', 'the', 'events', 'hope', 'i', 'might', 'write', 'about', 'them', 'not', 'because', 'im', 'actually', 'a', 'celebrity', 'i', 'left', 'that', 'morning', 'dressed', 'in', 'casual', 'clothes']
['running', 'the', 'events', 'hope', 'i', 'might', 'write', 'about', 'them', 'not', 'because', 'im', 'actually', 'a', 'celebrity', 'i', 'left', 'that', 'morning', 'dressed', 'in', 'casual', 'clothes', 'couldnt']
['the', 'events', 'hope', 'i', 'might', 'write', 'about', 'them', 'not', 'because', 'im', 'actually', 'a', 'celebrity', 'i', 'left', 'that', 'morning', 'dressed', 'in', 'casual', 'clothes', 'couldnt', 'get']
['events', 'hope', 'i', 'might', 'write', 'about', 'them', 'not', 'because', 'im', 'actually', 'a', 'celebrity', 'i', 'left', 'that', 'morning', 'dresse

['the', 'rest', 'of', 'the', 'time', 'and', 'bar', 'along', 'with', 'the', 'teams', 'that', 'holt', 'couldnt', 'have', 'considered', 'to', 'understand', 'as', 'much', 'as', 'the', 'patriots', 'is']
['the', 'rest', 'of', 'the', 'time', 'and', 'bar', 'along', 'with', 'the', 'teams', 'that', 'holt', 'couldnt', 'have', 'considered', 'to', 'understand', 'as', 'much', 'as', 'the', 'patriots', 'is', 'how']
['arent', 'with', 'their', 'careers', 'or', 'ever', 'puts', 'it', 'it', 'on', 'de', 'there', 'could', 'be', 'an', 'absolute', 'assassin', 'if', 'there', 'does', 'any', 'reason', 'why', 'this', 'is']
